# Fish Speech

### For Windows User / win用户

In [ ]:
!chcp 65001

### For Linux User / Linux 用户

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

### Prepare Model

In [1]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com 

!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5/

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 7 files:   0%|                                   | 0/7 [00:00<?, ?it/s]Still waiting to acquire lock on checkpoints/fish-speech-1.5/.cache/huggingface/download/firefly-gan-vq-fsq-8x1024-21hz-generator.pth.lock (elapsed: 10.0 seconds)
Still waiting to acquire lock on checkpoints/fish-speech-1.5/.cache/huggingface/download/model.pth.lock (elapsed: 10.0 seconds)
Fetching 7 files:  43%|███████████▌               | 3/7 [00:11<00:15,  3.92s/it]
^C
Traceback (most recent call last):
  File "/home/ca/.local/lib/python3.8/site-packages/tqdm/contrib/concurrent.py", line 51, in _executor_map
    return list(tqdm_class(ex.map(fn, *iterables, chunksize=chunksize), **kwargs))
  File "/home/ca/.local/lib/python3.8/site-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/lib/python3.8/concurrent/futures/_base.py", line 619, in result_iterator
    yield fs.pop().result()
  File "/u

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [ ]:
!python tools/run_webui.py \
    --llama-checkpoint-path checkpoints/fish-speech-1.5 \
    --decoder-checkpoint-path checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth \
    # --compile

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt: 

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python fish_speech/models/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [ ]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python fish_speech/models/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)